In [10]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from adjustText import adjust_text
import numpy as np

import nfl_data_py as nfl

In [11]:
# Turn off max columns for pandas DataFrame
pd.set_option('display.max_columns', None)

In [12]:
years = range(2018, 2023)

In [16]:
# Load top X recievers EPA/target
columns = ['season', 'posteam', 'season_type',
            'play', 'play_type', 'receiver_id', 'epa']

data = nfl.import_pbp_data(years, cache=True, alt_path='../cache', columns=columns)

data = data[columns]

2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [32]:
x = 50
minimum_targets = 50

In [33]:
top_x = data[(data['season_type'] == 'REG') & (data['play_type'] == 'pass') & (data['play'] == 1)]

In [34]:
# Group by season, posteam, receiver_id - get epa/target and total targets
top_x = top_x.groupby(['season', 'posteam', 'receiver_id']).agg({'epa': 'mean', 'play': 'count'}).reset_index()

top_x = top_x.rename(columns={'play': 'targets', 'epa': 'epa_target'})

In [35]:
# Get the top X receivers by epa/target
top_x = top_x[top_x['targets'] >= minimum_targets].sort_values('epa_target', ascending=False).head(x)

In [36]:
top_x

,season,posteam,receiver_id,epa_target,targets
458,2018,SEA,00-0032211,1.054917,71
1075,2020,ATL,00-0027944,0.785716,68
1263,2020,HOU,00-0033127,0.776056,75
43,2018,BAL,00-0034753,0.775205,50
1244,2020,GB,00-0033757,0.742266,59
295,2018,LAC,00-0033536,0.708766,68
1016,2019,TB,00-0033921,0.704218,122
1177,2020,CLE,00-0032977,0.694644,52
2034,2021,PHI,00-0034351,0.693291,77
111,2018,CIN,00-0033009,0.675895,108


In [37]:
charts = nfl.import_depth_charts(years=years)

charts = charts[['season', 'club_code', 'gsis_id', 'full_name', 'position']]

charts = charts.drop_duplicates()

charts = charts[charts['position'] == 'WR']

In [38]:
# Merge depth charts with passing data
df = top_x.merge(charts, left_on=['season', 'posteam', 'receiver_id'], right_on=['season', 'club_code', 'gsis_id'], how='inner')

In [39]:
df

,season,posteam,receiver_id,epa_target,targets,club_code,gsis_id,full_name,position
0,2018,SEA,00-0032211,1.054917,71,SEA,00-0032211,Tyler Lockett,WR
1,2020,ATL,00-0027944,0.785716,68,ATL,00-0027944,Julio Jones,WR
2,2020,HOU,00-0033127,0.776056,75,HOU,00-0033127,Will Fuller,WR
3,2018,LAC,00-0033536,0.708766,68,LAC,00-0033536,Mike Williams,WR
4,2019,TB,00-0033921,0.704218,122,TB,00-0033921,Chris Godwin,WR
5,2020,CLE,00-0032977,0.694644,52,CLE,00-0032977,Rashard Higgins,WR
6,2018,CIN,00-0033009,0.675895,108,CIN,00-0033009,Tyler Boyd,WR
7,2021,ARI,00-0030564,0.662156,64,ARI,00-0030564,DeAndre Hopkins,WR
8,2020,TB,00-0033921,0.657006,84,TB,00-0033921,Chris Godwin,WR
9,2018,LA,00-0033908,0.648917,55,LA,00-0033908,Cooper Kupp,WR


In [42]:
wins = nfl.import_schedules(years=years)

In [43]:
wins

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
5049,2018_01_ATL_PHI,2018,REG,1,2018-09-06,Thursday,20:20,ATL,12.0,PHI,18.0,Home,6.0,30.0,0.0,2018090600,57567.0,NaN,201809060phi,9338.0,401030710.0,7,7,-104.0,-106.0,1.0,-111.0,101.0,44.5,-101.0,-109.0,0,outdoors,grass,81.0,8.0,00-0026143,00-0029567,Matt Ryan,Nick Foles,Dan Quinn,Doug Pederson,John Hussey,PHI00,Lincoln Financial Field
5050,2018_01_BUF_BAL,2018,REG,1,2018-09-09,Sunday,13:00,BUF,3.0,BAL,47.0,Home,44.0,50.0,0.0,2018090900,57568.0,NaN,201809090rav,9346.0,401030711.0,7,7,309.0,-355.0,7.5,-106.0,-104.0,39.0,109.0,-120.0,0,outdoors,grass,61.0,12.0,00-0033958,00-0026158,Nathan Peterman,Joe Flacco,Sean McDermott,John Harbaugh,Shawn Hochuli,BAL00,M&T Bank Stadium
5051,2018_01_PIT_CLE,2018,REG,1,2018-09-09,Sunday,13:00,PIT,21.0,CLE,21.0,Home,0.0,42.0,1.0,2018090901,57569.0,NaN,201809090cle,9339.0,401030718.0,7,7,-173.0,156.0,-3.5,105.0,-116.0,41.0,-102.0,-108.0,1,outdoors,grass,58.0,11.0,00-0022924,00-0028118,Ben Roethlisberger,Tyrod Taylor,Mike Tomlin,Hue Jackson,Shawn Smith,CLE00,FirstEnergy Stadium
5052,2018_01_CIN_IND,2018,REG,1,2018-09-09,Sunday,13:00,CIN,34.0,IND,23.0,Home,-11.0,57.0,0.0,2018090902,57570.0,NaN,201809090clt,9340.0,401030717.0,7,7,-105.0,-105.0,-1.0,102.0,-113.0,47.5,100.0,-110.0,0,closed,fieldturf,NaN,NaN,00-0027973,00-0029668,Andy Dalton,Andrew Luck,Marvin Lewis,Frank Reich,Peter Morelli,IND00,Lucas Oil Stadium
5053,2018_01_TEN_MIA,2018,REG,1,2018-09-09,Sunday,13:00,TEN,20.0,MIA,27.0,Home,7.0,47.0,0.0,2018090903,57571.0,NaN,201809090mia,9341.0,401030716.0,7,7,-115.0,104.0,-1.0,-107.0,-103.0,43.5,-101.0,-109.0,0,outdoors,grass,89.0,7.0,00-0032268,00-0029701,Marcus Mariota,Ryan Tannehill,Mike Vrabel,Adam Gase,Jerome Boger,MIA00,Hard Rock Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6416,2022_20_CIN_BUF,2022,DIV,20,2023-01-22,Sunday,15:00,CIN,27.0,BUF,10.0,Home,-17.0,37.0,0.0,2023012200,NaN,NaN,202301220buf,NaN,401438007.0,7,7,226.0,-259.0,6.0,-107.0,-103.0,48.5,-107.0,-105.0,0,outdoors,a_turf,32.0,4.0,00-0036442,00-0034857,Joe Burrow,Josh Allen,Zac Taylor,Sean McDermott,Carl Cheffers,BUF00,New Era Field
6417,2022_20_DAL_SF,2022,DIV,20,2023-01-22,Sunday,18:30,DAL,12.0,SF,19.0,Home,7.0,31.0,0.0,2023012201,NaN,NaN,202301220sfo,NaN,401438006.0,6,8,164.0,-183.0,3.5,-107.0,-103.0,46.5,-105.0,-107.0,0,outdoors,grass,55.0,19.0,00-0033077,00-0037834,Dak Prescott,Brock Purdy,Mike McCarthy,Kyle Shanahan,Bill Vinovich,SFO01,Levi's Stadium
6418,2022_21_SF_PHI,2022,CON,21,2023-01-29,Sunday,15:00,SF,7.0,PHI,31.0,Home,24.0,38.0,0.0,2023012900,NaN,NaN,202301290phi,NaN,401438009.0,7,8,133.0,-147.0,2.5,105.0,-116.0,45.0,-114.0,101.0,0,outdoors,grass,52.0,14.0,00-0037834,00-0036389,Brock Purdy,Jalen Hurts,Kyle Shanahan,Nick Sirianni,John Hussey,PHI00,Lincoln Financial Field
6419,2022_21_CIN_KC,2022,CON,21,2023-01-29,Sunday,18:30,CIN,20.0,KC,23.0,Home,3.0,43.0,0.0,2023012901,NaN,NaN,202301290kan,NaN,401438008.0,7,8,119.0,-131.0,2.0,-103.0,-107.0,48.5,-106.0,-106.0,0,outdoors,grass,22.0,13.0,00-0036442,00-0033873,Joe Burrow,Patrick Mahomes,Zac Taylor,Andy Reid,Ron Torbert,KAN00,GEHA Field at Arrowhead Stadium
